In [ ]:
from depmapomics import tracker 
import pandas as pd
from genepy.utils import helper as h

In [ ]:
t = tracker.getTracker()

In [ ]:
for a in t.arxspan_id.unique():
    entries = t[t.arxspan_id == a]
    if "SANGER" in entries.source.tolist():
        sangerid = "MC-" + a[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True)
        sangeridx = t[(t.source == "SANGER") & (t.arxspan_id == a)].index
        t.loc[sangeridx, "model_condition_id"] = sangerid
    broadidx = t[(~t.source.isin(["SANGER"])) & (t.arxspan_id == a)].index
    broadid = "MC-" + a[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True)
    t.loc[broadidx, "model_condition_id"] = broadid

In [ ]:
t[t.model_condition_id.isnull()]

In [ ]:
check = [x[-4:] for x in t.model_condition_id.unique().tolist()]
len(check)

In [ ]:
len(set(check))

In [ ]:
from genepy.google.google_sheet import dfToSheet

dfToSheet(t, "Copy of ccle sample tracker - 030422", secret="../../.credentials.json")

In [ ]:
# t = tracker.getTracker()
for mc in t.model_condition_id.unique():
    entries = t[t.model_condition_id == mc]
    for d in entries.datatype.unique():
        idx = t[(t.model_condition_id == mc) & (t.datatype == d)].index
        t.loc[idx, 'profile_id'] = "PR-" + h.randomString(stringLength=6, stype="all", withdigits=True)

In [ ]:
t[t.profile_id.isnull()]

In [ ]:
len((t['model_condition_id'] + t['profile_id']).unique())

In [ ]:
len(t.profile_id.unique())

In [ ]:
dfToSheet(t, "Copy of ccle sample tracker - 030422", secret="../../.credentials.json")